In [3]:
#from pysocialwatcher import watcherAPI
import json
import pandas as pd
import ast


In [4]:
watcher = watcherAPI()
watcher.load_credentials_file("credentials.csv")

df = watcher.run_data_collection("./venezuelans_in_colombia_short.json")
#df = watcher.run_data_collection("./test.json")

NameError: name 'watcherAPI' is not defined

In [8]:
# alternatively, loads the already collected data:
#df = pd.read_csv("dataframe_collected_finished_1526497778_colombianstates.csv")
df1 = pd.read_csv("collections/sa/dataframe_collected_finished_1526830298.csv")
df2 = pd.read_csv("collections/colombia/dataframe_collected_finished_1526497778.csv")
df3 = pd.read_csv("collections/colombia/dataframe_collected_finished_1526595686.csv")
df4 = pd.read_csv("collections/colombia/dataframe_collected_finished_1526754549.csv")
#df["Country"] = "Colombia"

In [3]:
df.head()

NameError: name 'df' is not defined

In [6]:
def extract_relationship(d):
    if 1 in d:
        return "single"
    elif 2 in d:
        return "dating"
    elif 3 in d:
        return "married"
    else:
        return None

def extract_education(d):
    if d == [3, 7, 8, 9, 11]:
        return "graduated"
    elif d == [1, 12, 13]:
        return "no_degree"
    elif d == [2, 4, 5, 6, 10]:
        return "high_school"
    elif d == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        return "all"
    else:
        return None

def extract_group(d):
    id = d[0]["id"]
    if id == 6026404871583:
        return "Venezuelans"
    elif id == 6015559470583:
        return "All-expats"
    else:
        return None

def expand(row):
    place = None
    if "regions" in row["geo_locations"]:
        place = row["geo_locations"]["regions"][0]["name"]
    elif "countries" in row["geo_locations"]:
        place = "Country_" + row["geo_locations"]["countries"][0]
    elif "cities" in row["geo_locations"]:
        place = row["geo_locations"]["cities"][0]["name"]
    
    loctype = "_".join(row["geo_locations"]["location_types"])

    relationship, education, group = None, None, None
    for dimension in row["flexible_spec"]:
        if "relationship_statuses" in dimension:
            relationship = extract_relationship(dimension["relationship_statuses"])
        elif "education_statuses" in dimension:
            education = extract_education(dimension["education_statuses"])
        elif "behaviors" in dimension:
            group = extract_group(dimension["behaviors"])
            
    return row["age_min"], row["age_max"], place, loctype, row["genders"][0], relationship, education, group

def get_item(x):
    if not x:
        return None
    return x["name"]

#lambda x : expand(ast.literal_eval(x)) 
#df[["MinAge","MaxAge","Location","Gender","Relationship","Educaton","Group"]] = df["targeting"].apply(expand).apply(pd.Series)
#df[["MinAge","MaxAge","Location","LocationType","Gender","Relationship","Educaton","Group"]] = df["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)
#df["Group"] = df["citizenship"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
#df["Device"] = df["access_device"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))

In [9]:
df1[["MinAge","MaxAge","Location","LocationType","Gender","Relationship","Educaton","Group"]] = df1["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)
df2[["MinAge","MaxAge","Location","LocationType","Gender","Relationship","Educaton","Group"]] = df2["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)
df3[["MinAge","MaxAge","Location","LocationType","Gender","Relationship","Educaton","Group"]] = df3["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)
df4[["MinAge","MaxAge","Location","LocationType","Gender","Relationship","Educaton","Group"]] = df4["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)


In [6]:
#dfbr = df[df["Location"] == "BR"].copy()
#df = df[df["Location"] != "BR"].copy()

df[["Location","LocationType","MinAge","MaxAge","Gender","Group","dau_audience","mau_audience"]].sort_values(by=["Location","MinAge","MaxAge"]).to_csv("colombia.csv", index=False)


In [125]:
df[["Location","mau_audience","Gender"]].groupby("Gender")[::].mean()

In [ ]:
expats = df[df["Group"] == "Ex-pats (All)"][["Location","mau_audience"]]
venezuela = df[df["Group"] == "Expats (Venezuela)"][["Location","mau_audience"]]

merged = pd.merge(expats,venezuela, on=("Location"), suffixes=("_all","_ven"))
merged["ratio"] = merged["mau_audience_ven"] / merged["mau_audience_all"] 
merged

In [ ]:

expats = df[df["Group"] == "Ex-pats (All)"][["Device","Location","dau_audience"]]
venezuela = df[df["Group"] == "Expats (Venezuela)"][["Device","Location","dau_audience"]]

merged = pd.merge(expats,venezuela, on=("Location","Device"), suffixes=("_all","_ven"))
merged["ratio"] = merged["dau_audience_ven"] / merged["dau_audience_all"] 
merged


In [135]:
df["targeting"].head(1).values[0]

{'age_max': 24,
 'age_min': 18,
 'flexible_spec': [{'behaviors': [{'id': 6026404871583}]},
  {'education_statuses': [3, 7, 8, 9, 11]},
  {'relationship_statuses': [1, 11, 12, 13]}],
 'genders': [0],
 'geo_locations': {'location_types': [u'home'],
  u'regions': [{u'country_code': u'BR', u'key': u'438', u'name': u'Acre'}]},
 'publisher_platforms': ['facebook']}

In [105]:
dfs = {}
dfs[0] = (df[df["Gender"] == 0]).copy()
dfs[1] = (df[df["Gender"] == 1]).copy()
dfs[2] = (df[df["Gender"] == 2]).copy()

In [110]:
dfs[0]["mau_percentage"] = 1.0 * dfs[0]["mau_audience"] / dfs[0]["mau_audience"].sum()

In [117]:
dfs[0][["Gender","MinAge","MaxAge","Location","mau_audience","mau_percentage"]]

,Gender,MinAge,MaxAge,Location,mau_audience,mau_percentage
1,0,18,24.0,Acre,1000,0.009551
2,0,18,24.0,Alagoas,1000,0.009551
3,0,18,24.0,Amapá,1000,0.009551
4,0,18,24.0,Amazonas,2000,0.019102
5,0,18,24.0,Bahia,1000,0.009551
6,0,18,24.0,Ceará,1000,0.009551
7,0,18,24.0,Federal Distric,1000,0.009551
8,0,18,24.0,Espírito Santo,1000,0.009551
9,0,18,24.0,Mato Grosso do Sul,1000,0.009551
10,0,18,24.0,Maranhão,1000,0.009551


In [115]:
dfs[0]

,name,interests,ages_ranges,genders,behavior,scholarities,languages,family_statuses,relationship_statuses,geo_locations,...,response,dau_audience,mau_audience,timestamp,publisher_platforms,MinAge,MaxAge,Location,Gender,mau_percentage
1,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":0,""median_...",0,1000,1524998075,"[""facebook""]",18,24.0,Acre,0,0.009551
2,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":0,""median_...",0,1000,1524998075,"[""facebook""]",18,24.0,Alagoas,0,0.009551
3,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":0,""median_...",0,1000,1524998075,"[""facebook""]",18,24.0,Amapá,0,0.009551
4,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":22,""median...",1352,2000,1524998075,"[""facebook""]",18,24.0,Amazonas,0,0.019102
5,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":0,""median_...",0,1000,1524998075,"[""facebook""]",18,24.0,Bahia,0,0.009551
6,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":0,""median_...",0,1000,1524998075,"[""facebook""]",18,24.0,Ceará,0,0.009551
7,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":0,""median_...",0,1000,1524998075,"[""facebook""]",18,24.0,Federal Distric,0,0.009551
8,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":43,""median...",332,1000,1524998075,"[""facebook""]",18,24.0,Espírito Santo,0,0.009551
9,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":30,""median...",997,1000,1524998075,"[""facebook""]",18,24.0,Mato Grosso do Sul,0,0.009551
10,Test,NaN,"{u'max': 24, u'min': 18}",0,"{u'or': [6026404871583], u'name': u'Expats (Ve...",NaN,NaN,NaN,NaN,"{u'values': [{u'country_code': u'BR', u'key': ...",...,"{""data"":[{""bid_estimate"":{""min_bid"":33,""median...",498,1000,1524998075,"[""facebook""]",18,24.0,Maranhão,0,0.009551


In [17]:
df1[df1["Location"] == "Country_CO"][["mau_audience","dau_audience","Location","LocationType","Gender","Relationship","Educaton","Group"]]


,mau_audience,dau_audience,Location,LocationType,Gender,Relationship,Educaton,Group
0,910000,514210,Country_CO,home_recent,0,None,None,Venezuelans
1,1800000,968639,Country_CO,home_recent,0,None,None,All-expats
2,31000000,17988266,Country_CO,home_recent,0,None,None,None


In [23]:
df3[(df3["Location"] == "Country_CO") & (df2["Gender"] == 0) & (df2["MinAge"] == 13) & (df2["MaxAge"].isnull())][["mau_audience","dau_audience","Location","LocationType","Gender","Relationship","Educaton","Group"]]


,mau_audience,dau_audience,Location,LocationType,Gender,Relationship,Educaton,Group
450,72000,34009,Country_CO,home,0,None,None,Venezuelans
451,190000,80918,Country_CO,home,0,None,None,All-expats
452,3800000,1792586,Country_CO,home,0,None,None,None
453,72000,33817,Country_CO,rencent,0,None,None,Venezuelans
454,200000,81050,Country_CO,rencent,0,None,None,All-expats
455,3800000,1778399,Country_CO,rencent,0,None,None,None
456,72000,33417,Country_CO,home_recent,0,None,None,Venezuelans
457,200000,84052,Country_CO,home_recent,0,None,None,All-expats
458,3800000,1747999,Country_CO,home_recent,0,None,None,None


In [25]:
df3[(df3["Location"] == "Country_CO") & (df2["Gender"] == 0) & (df2["MinAge"] == 13) & (df2["MaxAge"].isnull())][["mau_audience","dau_audience","Location","LocationType","Gender","Relationship","Educaton","Group"]]

,mau_audience,dau_audience,Location,LocationType,Gender,Relationship,Educaton,Group
450,72000,34009,Country_CO,home,0,None,None,Venezuelans
451,190000,80918,Country_CO,home,0,None,None,All-expats
452,3800000,1792586,Country_CO,home,0,None,None,None
453,72000,33817,Country_CO,rencent,0,None,None,Venezuelans
454,200000,81050,Country_CO,rencent,0,None,None,All-expats
455,3800000,1778399,Country_CO,rencent,0,None,None,None
456,72000,33417,Country_CO,home_recent,0,None,None,Venezuelans
457,200000,84052,Country_CO,home_recent,0,None,None,All-expats
458,3800000,1747999,Country_CO,home_recent,0,None,None,None
